In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on
# %pycodestyle_off -to turn it off

In [2]:
# import all libraries
from minio import Minio
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, month, year
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, LongType
import requests
import json
import pandas as pd
from binance import Client
from datetime import datetime, date
import os
from dotenv import load_dotenv, dotenv_values
from binance.helpers import date_to_milliseconds, interval_to_milliseconds
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import time
load_dotenv()

True

6:80: E501 line too long (87 > 79 characters)


In [3]:
API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
MINIO_USER = os.getenv("MINIO_ROOT_USER")
MINIO_PASSWORD = os.getenv("MINIO_ROOT_PASSWORD")
client_binance = Client(API_KEY, SECRET_KEY)

In [4]:
spark = SparkSession.builder \
    .appName("CryptoETL") \
    .getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", MINIO_USER)#turn into access key in the future 
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", MINIO_PASSWORD)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")


'''
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")'''


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 20:39:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'\nsc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")\nsc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")\nsc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\nsc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")'

6:67: E261 at least two spaces before inline comment
6:67: E262 inline comment should start with '# '
6:80: E501 line too long (101 > 79 characters)
6:102: W291 trailing whitespace
12:80: E501 line too long (80 > 79 characters)
19:80: E501 line too long (90 > 79 characters)
21:1: W391 blank line at end of file


In [5]:
# tickers = client.get_all_tickers()
#client_binance.get_exchange_info() #has rate limit info
#client_binance.response.headers #has info on current usage

2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '


In [6]:
# start_datetime = datetime.strptime("01-01-2023", '%d-%m-%Y')
# start_date = start_datetime.strftime("%Y-%m-%dT%H:%M:%SZ")

# end_datetime = datetime.strptime("01-01-2024", '%d-%m-%Y')
# end_date_date = end_datetime.strftime("%Y-%m-%dT%H:%M:%SZ")

In [7]:
cryptos = ['BTCUSDT','ETHUSDT','LTCUSDT','BNBUSDT','DOGEUSDT']

1:21: E231 missing whitespace after ','
1:31: E231 missing whitespace after ','
1:41: E231 missing whitespace after ','
1:51: E231 missing whitespace after ','


In [39]:
klines = client_binance.get_historical_klines(
    symbol=symbol, interval=interval, start_str=start_date, end_str=end_date, limit=1000 
)

NameError: name 'interval' is not defined

2:80: E501 line too long (88 > 79 characters)
INFO:pycodestyle:2:80: E501 line too long (88 > 79 characters)
2:89: W291 trailing whitespace
INFO:pycodestyle:2:89: W291 trailing whitespace


In [7]:
#create a loop where start date,end date and output data are defined
#in loop says while end date is greater than start date, keep going
#check last value in output data, add last timeframe and use it as new start date
#if client_binance.response.headers['x-mbx-used-weight'] == 5000: sleep 30
#print used weight response head to monitor
#try and except that shows api error codes
#start_datetime = datetime.strptime(start_date, '%d-%m-%Y')
#end_datetime = datetime.strptime(end_date, '%d-%m-%Y')
#implement binance api exception
def get_data_monthly(symbol,interval,start_date,end_date):
    output = []
    try:
        while end_date > start_date:
            data = client_binance.get_historical_klines(
            symbol=symbol, interval=interval, start_str=start_date, end_str=end_date, limit=1000 
            )
            output +=data
            #timeframe = interval_to_milliseconds(interval)
            last_date = data[-1][0]
            #get month and year from last date and compare with end_date month and year
            last_datetime = datetime.utcfromtimestamp(last_date / 1000)
            end_datetime = datetime.strptime(end_date, '%d-%m-%Y')
            #the below is to check last data point
            if last_datetime.year == end_datetime.year and last_datetime.month == end_datetime.month:
                break
            start_datetime =  last_datetime + relativedelta(months=1)
            start_date = str(start_datetime)
            print(client_binance.response.headers['x-mbx-used-weight'])
            #for daily add sleep function
            if client_binance.response.headers['x-mbx-used-weight'] == 5000:
                time.sleep(10)
        return output
    except Exception as e:
        print("An exception occurred:", type(e).__name__, "–", e)


    

1:1: E265 block comment should start with '# '
2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '
3:80: E501 line too long (81 > 79 characters)
4:1: E265 block comment should start with '# '
5:1: E265 block comment should start with '# '
6:1: E265 block comment should start with '# '
7:1: E265 block comment should start with '# '
8:1: E265 block comment should start with '# '
9:1: E265 block comment should start with '# '
10:28: E231 missing whitespace after ','
10:37: E231 missing whitespace after ','
10:48: E231 missing whitespace after ','
15:13: E122 continuation line missing indentation or outdented
15:80: E501 line too long (96 > 79 characters)
15:97: W291 trailing whitespace
17:22: E225 missing whitespace around operator
18:13: E265 block comment should start with '# '
20:13: E265 block comment should start with '# '
20:80: E501 line too long (87 > 79 characters)
23:13: E265 block comment should start with '# '
24:80: E501 line too long

In [ ]:
# api response structure
"""[
  [
    1499040000000,      // Kline open time
    "0.01634790",       // Open price
    "0.80000000",       // High price
    "0.01575800",       // Low price
    "0.01577100",       // Close price
    "148976.11427815",  // Volume
    1499644799999,      // Kline Close time
    "2434.19055334",    // Quote asset volume
    308,                // Number of trades
    "1756.87402397",    // Taker buy base asset volume
    "28.46694368",      // Taker buy quote asset volume
    "0"                 // Unused field, ignore.
  ]
]"""

In [16]:
columns = [
    "Kline open time",
    "Open Price",
    "High price",
    "Low price",
    "Close Price",
    "Volume",
    "Kline Close time",
    "Quote asset volume",
    "Number of trades",
    "Taker buy base asset volume",
    "Taker buy quote asset volume",
    "Ignore",
]

In [13]:
def spark_df_monthly(output,columns):
    #implement schema check
    dataframe = spark.createDataFrame(output, columns)
    df = dataframe.withColumn("Kline open time", to_timestamp(col("Kline open time") / 1000))
    df = df.withColumnRenamed("Kline open time","datetime")
    df = df.select('datetime','Open Price','Close Price','Volume')
    df = df.withColumn("year", year(df["datetime"])) \
       .withColumn("month", month(df["datetime"]))
    return df

1:28: E231 missing whitespace after ','
2:5: E265 block comment should start with '# '
4:80: E501 line too long (93 > 79 characters)
5:48: E231 missing whitespace after ','
6:30: E231 missing whitespace after ','
6:43: E231 missing whitespace after ','
6:57: E231 missing whitespace after ','
8:8: E127 continuation line over-indented for visual indent


In [25]:
def upload_minio(minio_url,minio_user,minio_password,symbol,bucket_name,timeframe,df):
    pass
    client_minio = Minio(
    #"localhost:9000",  # Make sure you're using port 9000 for the S3 API
    minio_url,
    access_key = minio_user,
    secret_key = minio_password,
    secure=False  # Disable SSL if you're not using SSL certificates
    )
    df.write.mode("append") \
    .partitionBy("year", "month") \
    .parquet(f"s3a://{bucket_name}/{symbol}/{timeframe}")
    return f"Successfully {symbol} {timeframe} uploaded to minio"
    

1:27: E231 missing whitespace after ','
1:38: E231 missing whitespace after ','
1:53: E231 missing whitespace after ','
1:60: E231 missing whitespace after ','
1:72: E231 missing whitespace after ','
1:80: E501 line too long (86 > 79 characters)
1:82: E231 missing whitespace after ','
4:5: E122 continuation line missing indentation or outdented
4:5: E265 block comment should start with '# '
5:5: E122 continuation line missing indentation or outdented
6:5: E122 continuation line missing indentation or outdented
6:15: E251 unexpected spaces around keyword / parameter equals
6:17: E251 unexpected spaces around keyword / parameter equals
7:5: E122 continuation line missing indentation or outdented
7:15: E251 unexpected spaces around keyword / parameter equals
7:17: E251 unexpected spaces around keyword / parameter equals
8:5: E122 continuation line missing indentation or outdented
11:5: E122 continuation line missing indentation or outdented
12:5: E122 continuation line missing indentation

In [ ]:
#df.write.csv('test',header = True)

In [ ]:
#df.coalesce(1).write.csv("csv", header=True)
'''
destination_file = 'csv'
source_file = './csv/part-00000-403543db-3e05-4013-a90b-934dff0f9694-c000.csv' #just an object not technically a file
bucket_name = 'bucket1'
client_minio.fput_object(
        bucket_name, destination_file, source_file,
    )
'''

In [26]:
def minio_pipeline_monthly():
    columns = [
    "Kline open time",
    "Open Price",
    "High price",
    "Low price",
    "Close Price",
    "Volume",
    "Kline Close time",
    "Quote asset volume",
    "Number of trades",
    "Taker buy base asset volume",
    "Taker buy quote asset volume",
    "Ignore",
    ]
    data = get_data_monthly(symbol = symbol,interval = '1M',start_date = "01-01-2019",end_date = "31-12-2024")
    df_monthly = spark_df_monthly(data,columns)
    upload_minio(minio_url = 'localhost:9000',minio_user = MINIO_USER ,minio_password = MINIO_PASSWORD ,symbol = symbol ,bucket_name = 'bucket1',timeframe='Monthly',df = df_monthly)
    return "Successfully ran monthly pipeline to minio"
    #implement try and except, api codes from minio and binance
    #implement schema check

3:5: E122 continuation line missing indentation or outdented
4:5: E122 continuation line missing indentation or outdented
5:5: E122 continuation line missing indentation or outdented
6:5: E122 continuation line missing indentation or outdented
7:5: E122 continuation line missing indentation or outdented
8:5: E122 continuation line missing indentation or outdented
9:5: E122 continuation line missing indentation or outdented
10:5: E122 continuation line missing indentation or outdented
11:5: E122 continuation line missing indentation or outdented
12:5: E122 continuation line missing indentation or outdented
13:5: E122 continuation line missing indentation or outdented
14:5: E122 continuation line missing indentation or outdented
16:35: E251 unexpected spaces around keyword / parameter equals
16:37: E251 unexpected spaces around keyword / parameter equals
16:44: E231 missing whitespace after ','
16:53: E251 unexpected spaces around keyword / parameter equals
16:55: E251 unexpected spaces 

In [27]:
for symbol in cryptos:
    minio_pipeline_monthly()
    

25/03/11 12:43:19 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/03/11 12:43:21 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/11 12:43:21 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/11 12:43:21 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/11 12:43:21 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/11 12:43:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/11 12:43:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,7